In [2]:
import polars as pl
import matplotlib.pyplot as plt
df_preproc = pl.read_parquet("data/medicamentos_preprocesados.parquet")


In [3]:
import polars as pl
import matplotlib.pyplot as plt

def validar_relevancia_variables(df_preproc):
    """
    Analiza cada variable del dataset para determinar su relevancia 
    en el modelo de homologación de medicamentos.
    
    Args:
        df_preproc (pl.DataFrame): DataFrame preprocesado con medicamentos
        
    Returns:
        dict: Diccionario con análisis de relevancia por variable
        
    Notes:
        - Variables ID (CUM, EXPEDIENTE CUM, CONSECUTIVO) se analizan para bonus de expediente
        - Variables de estado (ESTADO REGISTRO, ESTADO CUM, MUESTRA MÉDICA) definen validez
        - Variables farmacológicas se evalúan por importancia en clustering
        - Se considera la especificidad farmacéutica de cada variable
        
    Examples:
        >>> relevancia = validar_relevancia_variables(df_preproc)
        >>> print(relevancia['PRINCIPIO ACTIVO']['decision'])
        'CRÍTICA - Variable principal para homologación'
    """
    
    print("="*80)
    print("🔍 VALIDACIÓN DE RELEVANCIA DE VARIABLES PARA MODELO DE HOMOLOGACIÓN")
    print("="*80)
    
    # Separar válidos e inválidos para análisis comparativo
    mascara_validos = (
        (df_preproc['ESTADO REGISTRO'] == 'Vigente') & 
        (df_preproc['ESTADO CUM'] == 'Activo') & 
        (df_preproc['MUESTRA MÉDICA'] == 'No')
    )
    
    validos = df_preproc.filter(mascara_validos)
    invalidos = df_preproc.filter(~mascara_validos)
    
    print(f"📊 Dataset: {len(validos):,} válidos, {len(invalidos):,} inválidos")
    
    # Diccionario para almacenar análisis de relevancia
    relevancia = {}
    
    # ========================================================================
    # 1. VARIABLES IDENTIFICADORAS
    # ========================================================================
    print(f"\n{'='*80}")
    print("🆔 ANÁLISIS DE VARIABLES IDENTIFICADORAS")
    print("="*80)
    
    # CUM (ID único)
    print(f"\n📋 VARIABLE: CUM")
    print("-" * 60)
    print("   🎯 Propósito: Identificador único del medicamento")
    print("   📊 Análisis:")
    
    cums_unicos = df_preproc['CUM'].n_unique()
    total_registros = len(df_preproc)
    print(f"      - CUMs únicos: {cums_unicos:,}")
    print(f"      - Total registros: {total_registros:,}")
    print(f"      - Ratio único: {cums_unicos/total_registros:.3f}")
    
    relevancia['CUM'] = {
        'tipo': 'IDENTIFICADOR',
        'decision': 'NO USAR - Variable ID, no aporta similitud farmacológica',
        'justificacion': 'Identificador único sin información farmacológica',
        'uso_recomendado': 'Usar solo para referenciar resultado de homologación'
    }
    
    # EXPEDIENTE CUM + CONSECUTIVO
    print(f"\n📋 VARIABLE: EXPEDIENTE CUM + CONSECUTIVO")
    print("-" * 60)
    print("   🎯 Propósito: Agrupar medicamentos del mismo expediente")
    
    # Análisis de expedientes
    expedientes_validos = validos.group_by('EXPEDIENTE CUM').agg([
        pl.len().alias('medicamentos_en_expediente'),
        pl.col('PRINCIPIO ACTIVO').n_unique().alias('principios_diferentes')
    ]).sort('medicamentos_en_expediente', descending=True)
    
    expedientes_multiples = expedientes_validos.filter(pl.col('medicamentos_en_expediente') > 1)
    
    print(f"   📊 Análisis de expedientes válidos:")
    print(f"      - Expedientes únicos: {len(expedientes_validos):,}")
    print(f"      - Expedientes con múltiples medicamentos: {len(expedientes_multiples):,}")
    
    if len(expedientes_multiples) > 0:
        print(f"      - Promedio medicamentos por expediente múltiple: {expedientes_multiples['medicamentos_en_expediente'].mean():.1f}")
        
        # Analizar si medicamentos del mismo expediente tienen el mismo principio activo
        mismo_principio = expedientes_multiples.filter(pl.col('principios_diferentes') == 1)
        print(f"      - Expedientes con MISMO principio activo: {len(mismo_principio):,} ({len(mismo_principio)/len(expedientes_multiples)*100:.1f}%)")
        
        if len(mismo_principio) > 0:
            relevancia['EXPEDIENTE CUM'] = {
                'tipo': 'BONUS',
                'decision': 'USAR COMO BONUS - Medicamentos del mismo expediente son más similares',
                'justificacion': f'{len(mismo_principio)/len(expedientes_multiples)*100:.1f}% de expedientes múltiples tienen mismo principio activo',
                'uso_recomendado': 'Bonus de similitud si coincide expediente'
            }
        else:
            relevancia['EXPEDIENTE CUM'] = {
                'tipo': 'IRRELEVANTE',
                'decision': 'NO USAR - Expedientes no agrupan medicamentos similares',
                'justificacion': 'Medicamentos del mismo expediente tienen principios activos diferentes',
                'uso_recomendado': 'Ignorar en modelo'
            }
    else:
        relevancia['EXPEDIENTE CUM'] = {
            'tipo': 'IRRELEVANTE',
            'decision': 'NO USAR - Todos los expedientes son únicos',
            'justificacion': 'No hay agrupación real de medicamentos por expediente',
            'uso_recomendado': 'Ignorar en modelo'
        }
    
    # ========================================================================
    # 2. VARIABLES DE ESTADO (YA SABEMOS QUE DEFINEN VALIDEZ)
    # ========================================================================
    print(f"\n{'='*80}")
    print("📋 VARIABLES DE ESTADO")
    print("="*80)
    
    for var_estado in ['ESTADO REGISTRO', 'ESTADO CUM', 'MUESTRA MÉDICA']:
        relevancia[var_estado] = {
            'tipo': 'FILTRO',
            'decision': 'USAR PARA FILTRAR - Define medicamentos válidos',
            'justificacion': 'Criterio de validez, no de similitud',
            'uso_recomendado': 'Filtro previo, no incluir en clustering'
        }
    
    print("   ✅ Variables de estado: USAR SOLO PARA FILTRAR (no para clustering)")
    
    # ========================================================================
    # 3. VARIABLES FARMACOLÓGICAS PRINCIPALES
    # ========================================================================
    print(f"\n{'='*80}")
    print("🧬 ANÁLISIS DE VARIABLES FARMACOLÓGICAS")
    print("="*80)
    
    # PRINCIPIO ACTIVO
    print(f"\n📋 VARIABLE: PRINCIPIO ACTIVO")
    print("-" * 60)
    
    principios_validos = validos['PRINCIPIO ACTIVO'].n_unique()
    principios_invalidos = invalidos['PRINCIPIO ACTIVO'].n_unique()
    principios_comunes = len(set(validos['PRINCIPIO ACTIVO'].unique().to_list()).intersection(
        set(invalidos['PRINCIPIO ACTIVO'].unique().to_list())
    ))
    
    print(f"   📊 Análisis de cobertura:")
    print(f"      - Principios únicos en válidos: {principios_validos:,}")
    print(f"      - Principios únicos en inválidos: {principios_invalidos:,}")
    print(f"      - Principios comunes (homologables): {principios_comunes:,}")
    print(f"      - Cobertura: {principios_comunes/principios_invalidos*100:.1f}%")
    
    relevancia['PRINCIPIO ACTIVO'] = {
        'tipo': 'CRÍTICA',
        'decision': 'OBLIGATORIA - Variable principal para homologación',
        'justificacion': f'Esencia química del medicamento. Cobertura {principios_comunes/principios_invalidos*100:.1f}%',
        'uso_recomendado': 'Filtro duro: solo homologar medicamentos con mismo principio activo'
    }
    
    # ATC (ya analizado anteriormente)
    print(f"\n📋 VARIABLE: ATC")
    print("-" * 60)
    print("   📊 Análisis previo mostró: 90.2% principios tienen UN SOLO ATC")
    
    relevancia['ATC'] = {
        'tipo': 'OPCIONAL',
        'decision': 'OPCIONAL - Redundante con Principio Activo en 90% casos',
        'justificacion': 'Solo 9.8% de principios tienen múltiples ATC',
        'uso_recomendado': 'No incluir en filtros duros, usar como feature adicional'
    }
    
    # DESCRIPCIÓN_ATC
    relevancia['DESCRIPCIÓN_ATC'] = {
        'tipo': 'REDUNDANTE',
        'decision': 'NO USAR - Redundante con código ATC',
        'justificacion': 'Información descriptiva, el código ATC es suficiente',
        'uso_recomendado': 'Ignorar en modelo'
    }
    
    # VÍA ADMINISTRACIÓN
    print(f"\n📋 VARIABLE: VÍA ADMINISTRACIÓN")
    print("-" * 60)
    
    vias_validos = validos['VÍA ADMINISTRACIÓN'].value_counts().sort("count", descending=True)
    vias_invalidos = invalidos['VÍA ADMINISTRACIÓN'].value_counts().sort("count", descending=True)
    
    print(f"   📊 Análisis de vías:")
    print(f"      - Vías únicas en válidos: {validos['VÍA ADMINISTRACIÓN'].n_unique()}")
    print(f"      - Vías únicas en inválidos: {invalidos['VÍA ADMINISTRACIÓN'].n_unique()}")
    print(f"      - Vía más común en válidos: {vias_validos[0, 'VÍA ADMINISTRACIÓN']} ({vias_validos[0, 'count']:,} casos)")
    
    # Verificar cobertura de vías
    vias_validas = set(validos['VÍA ADMINISTRACIÓN'].unique().to_list())
    vias_invalidas = set(invalidos['VÍA ADMINISTRACIÓN'].unique().to_list())
    vias_comunes = vias_validas.intersection(vias_invalidas)
    
    print(f"      - Cobertura de vías: {len(vias_comunes)}/{len(vias_invalidas)} ({len(vias_comunes)/len(vias_invalidas)*100:.1f}%)")
    
    relevancia['VÍA ADMINISTRACIÓN'] = {
        'tipo': 'CRÍTICA',
        'decision': 'OBLIGATORIA - Determina uso clínico del medicamento',
        'justificacion': 'Cambia fundamentalmente la aplicación del medicamento',
        'uso_recomendado': 'Filtro duro: solo homologar medicamentos con misma vía'
    }
    
    # FORMA FARMACÉUTICA
    print(f"\n📋 VARIABLE: FORMA FARMACÉUTICA")
    print("-" * 60)
    
    formas_validos = validos['FORMA FARMACÉUTICA'].n_unique()
    formas_invalidos = invalidos['FORMA FARMACÉUTICA'].n_unique()
    
    # Análisis de flexibilidad entre formas
    formas_validas = set(validos['FORMA FARMACÉUTICA'].unique().to_list())
    formas_invalidas = set(invalidos['FORMA FARMACÉUTICA'].unique().to_list())
    formas_comunes = formas_validas.intersection(formas_invalidas)
    
    print(f"   📊 Análisis de formas:")
    print(f"      - Formas únicas en válidos: {formas_validos}")
    print(f"      - Formas únicas en inválidos: {formas_invalidos}")
    print(f"      - Cobertura de formas: {len(formas_comunes)}/{len(formas_invalidas)} ({len(formas_comunes)/len(formas_invalidas)*100:.1f}%)")
    
    # Analizar combinaciones PA + VÍA con múltiples formas
    combinaciones_multiples_formas = validos.group_by(['PRINCIPIO ACTIVO', 'VÍA ADMINISTRACIÓN']).agg([
        pl.col('FORMA FARMACÉUTICA').n_unique().alias('formas_diferentes'),
        pl.len().alias('medicamentos')
    ]).filter(pl.col('formas_diferentes') > 1)
    
    print(f"      - Combinaciones PA+Vía con múltiples formas: {len(combinaciones_multiples_formas):,}")
    
    if len(combinaciones_multiples_formas) > 0:
        relevancia['FORMA FARMACÉUTICA'] = {
            'tipo': 'IMPORTANTE',
            'decision': 'USAR PARA RANKING - Preferir misma forma, permitir diferentes',
            'justificacion': f'{len(combinaciones_multiples_formas):,} combinaciones PA+Vía tienen múltiples formas disponibles',
            'uso_recomendado': 'No filtro duro, usar para scoring/ranking de similitud'
        }
    else:
        relevancia['FORMA FARMACÉUTICA'] = {
            'tipo': 'CRÍTICA',
            'decision': 'OBLIGATORIA - Cada PA+Vía tiene forma única',
            'justificacion': 'No hay flexibilidad en formas farmacéuticas',
            'uso_recomendado': 'Incluir en filtro duro'
        }
    
    # ========================================================================
    # 4. VARIABLES DE DOSIFICACIÓN
    # ========================================================================
    print(f"\n{'='*80}")
    print("💊 ANÁLISIS DE VARIABLES DE DOSIFICACIÓN")
    print("="*80)
    
    # CONCENTRACIÓN
    print(f"\n📋 VARIABLE: CONCENTRACIÓN")
    print("-" * 60)
    
    concentraciones = validos.filter(pl.col('CONCENTRACIÓN').is_not_null())['CONCENTRACIÓN'].unique()
    print(f"   📊 Valores únicos de concentración: {len(concentraciones)}")
    
    if len(concentraciones) > 0:
        # Mostrar ejemplos
        ejemplos = concentraciones.head(10).to_list()
        print(f"   📝 Ejemplos: {ejemplos}")
        
        # Verificar si son solo letras
        solo_letras = all(isinstance(x, str) and len(x) == 1 and x.isalpha() for x in ejemplos if x is not None)
        
        if solo_letras:
            relevancia['CONCENTRACIÓN'] = {
                'tipo': 'DUDOSA',
                'decision': 'INVESTIGAR MEJOR - Solo letras sin contexto claro',
                'justificacion': 'Valores son letras individuales, significado unclear',
                'uso_recomendado': 'Omitir hasta clarificar significado'
            }
        else:
            relevancia['CONCENTRACIÓN'] = {
                'tipo': 'IMPORTANTE',
                'decision': 'USAR PARA RANKING - Información de concentración relevante',
                'justificacion': 'Concentración afecta dosificación y equivalencia',
                'uso_recomendado': 'Feature para scoring, no filtro duro'
            }
    else:
        relevancia['CONCENTRACIÓN'] = {
            'tipo': 'NO_DISPONIBLE',
            'decision': 'NO USAR - Sin datos de concentración',
            'justificacion': 'Campo vacío o sin información útil',
            'uso_recomendado': 'Ignorar en modelo'
        }
    
    # UNIDAD MEDIDA
    print(f"\n📋 VARIABLE: UNIDAD MEDIDA")
    print("-" * 60)
    
    unidades_medida = validos['UNIDAD MEDIDA'].value_counts().sort("count", descending=True)
    print(f"   📊 Unidades únicas: {validos['UNIDAD MEDIDA'].n_unique()}")
    print(f"   🏆 Top 3 unidades: {unidades_medida.head(3).select(['UNIDAD MEDIDA', 'count']).to_pandas().to_string(index=False)}")
    
    relevancia['UNIDAD MEDIDA'] = {
        'tipo': 'IMPORTANTE',
        'decision': 'USAR PARA RANKING - Complementa información de dosificación',
        'justificacion': 'Unidad de medida es crucial para equivalencia farmacológica',
        'uso_recomendado': 'Feature para scoring, bonus por unidad igual'
    }
    
    # CANTIDAD
    print(f"\n📋 VARIABLE: CANTIDAD")
    print("-" * 60)
    
    cantidad_stats = validos['CANTIDAD'].describe()
    cantidad_nulos = validos['CANTIDAD'].null_count()
    
    print(f"   📊 Estadísticas de cantidad:")
    print(f"      - Valores nulos: {cantidad_nulos:,} ({cantidad_nulos/len(validos)*100:.1f}%)")
    if cantidad_nulos < len(validos):
        print(f"      - Promedio: {validos['CANTIDAD'].mean():.2f}")
        print(f"      - Mediana: {validos['CANTIDAD'].median():.2f}")
        print(f"      - Rango: {validos['CANTIDAD'].min():.2f} - {validos['CANTIDAD'].max():.2f}")
    
    relevancia['CANTIDAD'] = {
        'tipo': 'OPCIONAL',
        'decision': 'USAR PARA RANKING - Información de presentación',
        'justificacion': 'Cantidad puede indicar presentación similar',
        'uso_recomendado': 'Feature secundaria para scoring'
    }
    
    # CANTIDAD CUM
    print(f"\n📋 VARIABLE: CANTIDAD CUM")
    print("-" * 60)
    
    cantidad_cum_stats = validos['CANTIDAD CUM'].describe()
    cantidad_cum_nulos = validos['CANTIDAD CUM'].null_count()
    
    print(f"   📊 Estadísticas de cantidad CUM:")
    print(f"      - Valores nulos: {cantidad_cum_nulos:,} ({cantidad_cum_nulos/len(validos)*100:.1f}%)")
    
    # Verificar correlación con CANTIDAD
    if cantidad_nulos < len(validos) * 0.5 and cantidad_cum_nulos < len(validos) * 0.5:
        # Ambas variables tienen datos suficientes
        correlacion_pearson = validos.select([
            pl.corr('CANTIDAD', 'CANTIDAD CUM').alias('correlacion')
        ]).item(0, 'correlacion')
        
        print(f"      - Correlación con CANTIDAD: {correlacion_pearson:.3f}")
        
        if abs(correlacion_pearson) > 0.8:
            relevancia['CANTIDAD CUM'] = {
                'tipo': 'REDUNDANTE',
                'decision': 'NO USAR - Redundante con CANTIDAD',
                'justificacion': f'Alta correlación ({correlacion_pearson:.3f}) con CANTIDAD',
                'uso_recomendado': 'Usar solo CANTIDAD'
            }
        else:
            relevancia['CANTIDAD CUM'] = {
                'tipo': 'OPCIONAL',
                'decision': 'USAR COMO ALTERNATIVA - Información complementaria',
                'justificacion': f'Correlación moderada ({correlacion_pearson:.3f}) con CANTIDAD',
                'uso_recomendado': 'Feature adicional si aporta información única'
            }
    else:
        relevancia['CANTIDAD CUM'] = {
            'tipo': 'OPCIONAL',
            'decision': 'EVALUAR DISPONIBILIDAD - Muchos valores nulos',
            'justificacion': f'{cantidad_cum_nulos/len(validos)*100:.1f}% de valores nulos',
            'uso_recomendado': 'Usar solo si tiene mejor cobertura que CANTIDAD'
        }
    
    # UNIDAD REFERENCIA + UNIDAD
    for var_unidad in ['UNIDAD REFERENCIA', 'UNIDAD']:
        unidades_unicas = validos[var_unidad].n_unique()
        unidades_nulos = validos[var_unidad].null_count()
        
        print(f"\n📋 VARIABLE: {var_unidad}")
        print("-" * 60)
        print(f"   📊 Valores únicos: {unidades_unicas}")
        print(f"   📊 Valores nulos: {unidades_nulos:,} ({unidades_nulos/len(validos)*100:.1f}%)")
        
        if unidades_nulos < len(validos) * 0.3:  # Menos del 30% nulos
            relevancia[var_unidad] = {
                'tipo': 'OPCIONAL',
                'decision': 'USAR PARA RANKING - Información de unidades',
                'justificacion': f'Información de unidades con {unidades_nulos/len(validos)*100:.1f}% nulos',
                'uso_recomendado': 'Feature secundaria para scoring'
            }
        else:
            relevancia[var_unidad] = {
                'tipo': 'BAJA_CALIDAD',
                'decision': 'NO USAR - Demasiados valores nulos',
                'justificacion': f'{unidades_nulos/len(validos)*100:.1f}% de valores nulos',
                'uso_recomendado': 'Ignorar por baja calidad de datos'
            }
    
    # ========================================================================
    # 5. RESUMEN Y RECOMENDACIONES
    # ========================================================================
    print(f"\n{'='*80}")
    print("🎯 RESUMEN DE RELEVANCIA DE VARIABLES")
    print("="*80)
    
    # Agrupar por tipo de decisión
    criticas = []
    importantes = []
    opcionales = []
    no_usar = []
    
    for variable, info in relevancia.items():
        if info['tipo'] in ['CRÍTICA']:
            criticas.append(variable)
        elif info['tipo'] in ['IMPORTANTE']:
            importantes.append(variable)
        elif info['tipo'] in ['OPCIONAL', 'BONUS']:
            opcionales.append(variable)
        else:
            no_usar.append(variable)
    
    print(f"\n🔥 VARIABLES CRÍTICAS (obligatorias para homologación):")
    for var in criticas:
        print(f"   ✅ {var}: {relevancia[var]['decision']}")
    
    print(f"\n⭐ VARIABLES IMPORTANTES (usar para ranking):")
    for var in importantes:
        print(f"   🔶 {var}: {relevancia[var]['decision']}")
    
    print(f"\n📋 VARIABLES OPCIONALES (features adicionales):")
    for var in opcionales:
        print(f"   🔸 {var}: {relevancia[var]['decision']}")
    
    print(f"\n❌ VARIABLES A NO USAR:")
    for var in no_usar:
        print(f"   ⭕ {var}: {relevancia[var]['decision']}")
    
    # Estrategia recomendada
    print(f"\n💡 ESTRATEGIA RECOMENDADA PARA MODELO:")
    print("─" * 60)
    print("1. FILTRO DURO:")
    filtro_duro = [var for var in criticas if relevancia[var]['uso_recomendado'].startswith('Filtro duro')]
    print(f"   {' + '.join(filtro_duro)}")
    
    print("\n2. SCORING/RANKING:")
    scoring_vars = [var for var in importantes + opcionales 
                   if 'scoring' in relevancia[var]['uso_recomendado'].lower() or 
                      'ranking' in relevancia[var]['uso_recomendado'].lower()]
    print(f"   {', '.join(scoring_vars)}")
    
    print("\n3. FEATURES BONUS:")
    bonus_vars = [var for var in opcionales if relevancia[var]['tipo'] == 'BONUS']
    if bonus_vars:
        print(f"   {', '.join(bonus_vars)}")
    else:
        print("   Ninguna")
    
    return relevancia

# Uso del código:
# relevancia_variables = validar_relevancia_variables(df_preproc)

In [7]:
validar_relevancia_variables(df_preproc)

🔍 VALIDACIÓN DE RELEVANCIA DE VARIABLES PARA MODELO DE HOMOLOGACIÓN
📊 Dataset: 93,140 válidos, 313,667 inválidos

🆔 ANÁLISIS DE VARIABLES IDENTIFICADORAS

📋 VARIABLE: CUM
------------------------------------------------------------
   🎯 Propósito: Identificador único del medicamento
   📊 Análisis:
      - CUMs únicos: 155,341
      - Total registros: 406,807
      - Ratio único: 0.382

📋 VARIABLE: EXPEDIENTE CUM + CONSECUTIVO
------------------------------------------------------------
   🎯 Propósito: Agrupar medicamentos del mismo expediente
   📊 Análisis de expedientes válidos:
      - Expedientes únicos: 9,389
      - Expedientes con múltiples medicamentos: 8,311
      - Promedio medicamentos por expediente múltiple: 11.1
      - Expedientes con MISMO principio activo: 6,358 (76.5%)

📋 VARIABLES DE ESTADO
   ✅ Variables de estado: USAR SOLO PARA FILTRAR (no para clustering)

🧬 ANÁLISIS DE VARIABLES FARMACOLÓGICAS

📋 VARIABLE: PRINCIPIO ACTIVO
----------------------------------------

{'CUM': {'tipo': 'IDENTIFICADOR',
  'decision': 'NO USAR - Variable ID, no aporta similitud farmacológica',
  'justificacion': 'Identificador único sin información farmacológica',
  'uso_recomendado': 'Usar solo para referenciar resultado de homologación'},
 'EXPEDIENTE CUM': {'tipo': 'BONUS',
  'decision': 'USAR COMO BONUS - Medicamentos del mismo expediente son más similares',
  'justificacion': '76.5% de expedientes múltiples tienen mismo principio activo',
  'uso_recomendado': 'Bonus de similitud si coincide expediente'},
 'ESTADO REGISTRO': {'tipo': 'FILTRO',
  'decision': 'USAR PARA FILTRAR - Define medicamentos válidos',
  'justificacion': 'Criterio de validez, no de similitud',
  'uso_recomendado': 'Filtro previo, no incluir en clustering'},
 'ESTADO CUM': {'tipo': 'FILTRO',
  'decision': 'USAR PARA FILTRAR - Define medicamentos válidos',
  'justificacion': 'Criterio de validez, no de similitud',
  'uso_recomendado': 'Filtro previo, no incluir en clustering'},
 'MUESTRA MÉDICA'

In [8]:
# Análisis rápido de cobertura ATC
def comparar_cobertura_atc_vs_principio(df_preproc):
    mascara_validos = (
        (df_preproc['ESTADO REGISTRO'] == 'Vigente') & 
        (df_preproc['ESTADO CUM'] == 'Activo') & 
        (df_preproc['MUESTRA MÉDICA'] == 'No')
    )
    
    validos = df_preproc.filter(mascara_validos)
    invalidos = df_preproc.filter(~mascara_validos)
    
    print("🔍 COMPARACIÓN DE COBERTURA: ATC vs PRINCIPIO ACTIVO")
    print("="*70)
    
    # Cobertura por Principio Activo (ya sabemos)
    principios_validos = set(validos['PRINCIPIO ACTIVO'].unique().to_list())
    principios_invalidos = set(invalidos['PRINCIPIO ACTIVO'].unique().to_list())
    principios_comunes = principios_validos.intersection(principios_invalidos)
    
    # Cobertura por ATC
    atc_validos = set(validos['ATC'].unique().to_list())
    atc_invalidos = set(invalidos['ATC'].unique().to_list())
    atc_comunes = atc_validos.intersection(atc_invalidos)
    
    print(f"📊 COBERTURA POR PRINCIPIO ACTIVO:")
    print(f"   Principios en válidos: {len(principios_validos):,}")
    print(f"   Principios en inválidos: {len(principios_invalidos):,}")
    print(f"   Principios comunes: {len(principios_comunes):,}")
    print(f"   🎯 COBERTURA: {len(principios_comunes)/len(principios_invalidos)*100:.1f}%")
    
    print(f"\n📊 COBERTURA POR ATC:")
    print(f"   ATC en válidos: {len(atc_validos):,}")
    print(f"   ATC en inválidos: {len(atc_invalidos):,}")
    print(f"   ATC comunes: {len(atc_comunes):,}")
    print(f"   🎯 COBERTURA: {len(atc_comunes)/len(atc_invalidos)*100:.1f}%")
    
    # Medicamentos inválidos cubiertos por ATC
    invalidos_cubiertos_atc = invalidos.filter(
        pl.col('ATC').is_in(list(atc_comunes))
    )
    
    print(f"\n📋 MEDICAMENTOS INVÁLIDOS CUBIERTOS:")
    print(f"   Por Principio Activo: {len(invalidos.filter(pl.col('PRINCIPIO ACTIVO').is_in(list(principios_comunes)))):,}")
    print(f"   Por ATC: {len(invalidos_cubiertos_atc):,}")
    print(f"   🎯 MEJORA CON ATC: {len(invalidos_cubiertos_atc)/len(invalidos)*100:.1f}% vs {len(principios_comunes)/len(principios_invalidos)*100:.1f}%")
    
    return {
        'cobertura_principios': len(principios_comunes)/len(principios_invalidos)*100,
        'cobertura_atc': len(atc_comunes)/len(atc_invalidos)*100,
        'medicamentos_cubiertos_principios': len(invalidos.filter(pl.col('PRINCIPIO ACTIVO').is_in(list(principios_comunes)))),
        'medicamentos_cubiertos_atc': len(invalidos_cubiertos_atc)
    }

# Ejecutar
comparar_cobertura_atc_vs_principio(df_preproc)

🔍 COMPARACIÓN DE COBERTURA: ATC vs PRINCIPIO ACTIVO
📊 COBERTURA POR PRINCIPIO ACTIVO:
   Principios en válidos: 6,075
   Principios en inválidos: 18,731
   Principios comunes: 4,485
   🎯 COBERTURA: 23.9%

📊 COBERTURA POR ATC:
   ATC en válidos: 1,568
   ATC en inválidos: 2,303
   ATC comunes: 1,452
   🎯 COBERTURA: 63.0%

📋 MEDICAMENTOS INVÁLIDOS CUBIERTOS:
   Por Principio Activo: 186,236
   Por ATC: 287,880
   🎯 MEJORA CON ATC: 91.8% vs 23.9%


{'cobertura_principios': 23.944263520367308,
 'cobertura_atc': 63.0481980026053,
 'medicamentos_cubiertos_principios': 186236,
 'medicamentos_cubiertos_atc': 287880}

In [ ]:
# Crear un sample variado del dataframe
sample_variado = df_preproc.sample(n=1000, seed=42)

# Mostrar información básica del sample
print("📊 SAMPLE VARIADO DEL DATASET")
print("="*50)
print(f"Tamaño del sample: {len(sample_variado):,} registros")
print(f"Dataset completo: {len(df_preproc):,} registros")
print(f"Proporción: {len(sample_variado)/len(df_preproc)*100:.1f}%")

# Verificar variedad en variables clave
print(f"\n🔍 VARIEDAD EN EL SAMPLE:")
print(f"   Estados registro únicos: {sample_variado['ESTADO REGISTRO'].n_unique()}")
print(f"   Principios activos únicos: {sample_variado['PRINCIPIO ACTIVO'].n_unique()}")
print(f"   Vías administración únicas: {sample_variado['VÍA ADMINISTRACIÓN'].n_unique()}")
print(f"   Formas farmacéuticas únicas: {sample_variado['FORMA FARMACÉUTICA'].n_unique()}")

# Mostrar el sample
sample_variado

In [4]:
df_preproc = pl.read_parquet("data/medicamentos_preprocesados.parquet")

In [10]:
df_preproc.filter(pl.col('CANTIDAD CUM') > 200000)

CUM,EXPEDIENTE CUM,CONSECUTIVO,ESTADO REGISTRO,CANTIDAD CUM,ESTADO CUM,MUESTRA MÉDICA,UNIDAD,ATC,DESCRIPCIÓN_ATC,VÍA ADMINISTRACIÓN,CONCENTRACIÓN,PRINCIPIO ACTIVO,UNIDAD MEDIDA,CANTIDAD,UNIDAD REFERENCIA,FORMA FARMACÉUTICA
str,i64,i64,str,f64,str,str,str,str,str,str,str,str,str,f64,str,str
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""CLORURO DE MAGNESIO HEXAHIDRAT…","""g""",0.357,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""ACIDO ACETICO GLACIAL""","""g""",0.63,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""ACIDO ACETICO GLACIAL""","""g""",0.63,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""CLORURO DE CALCIO DIHIDRATADO""","""g""",0.644,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""CLORURO DE CALCIO DIHIDRATADO""","""g""",0.644,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""CLORURO DE SODIO""","""g""",21.067,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""CLORURO DE SODIO""","""g""",21.067,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""
"""19935495-2""",19935495,2,"""Vencido""",208000.0,"""Inactivo""","""No""","""ml""","""B05ZA98""","""CONCENTRADO PARA HEMODIALISIS""","""PARENTERAL""","""A""","""CLORURO DE POTASIO""","""g""",0.522,"""100 ML DE SOLUCIÓN PARA HEMODI…","""SUSPENSION INYECTABLE"""


In [12]:
# Encontrar CUM duplicados
cum_duplicados = df_preproc.group_by('CUM').agg([
    pl.len().alias('cantidad_registros')
]).filter(
    pl.col('cantidad_registros') > 1
).sort('cantidad_registros', descending=True)

print(f"📊 CUM DUPLICADOS:")
print(f"Total CUM únicos: {df_preproc['CUM'].n_unique():,}")
print(f"Total registros: {len(df_preproc):,}")
print(f"CUM duplicados: {len(cum_duplicados):,}")

# Mostrar los CUM más duplicados
print(f"\n🔍 TOP 10 CUM MÁS DUPLICADOS:")
print(cum_duplicados.head(10))

# Ver ejemplos de registros duplicados
print(f"\n📋 EJEMPLO DE REGISTROS DUPLICADOS:")
primer_cum_duplicado = cum_duplicados[0, 'CUM']
registros_ejemplo = df_preproc.filter(pl.col('CUM') == primer_cum_duplicado)
print(f"CUM: {primer_cum_duplicado} ({len(registros_ejemplo)} registros)")
registros_ejemplo

📊 CUM DUPLICADOS:
Total CUM únicos: 155,341
Total registros: 406,807
CUM duplicados: 95,566

🔍 TOP 10 CUM MÁS DUPLICADOS:
shape: (10, 2)
┌─────────────┬────────────────────┐
│ CUM         ┆ cantidad_registros │
│ ---         ┆ ---                │
│ str         ┆ u32                │
╞═════════════╪════════════════════╡
│ 19903212-1  ┆ 232                │
│ 19908011-3  ┆ 228                │
│ 19908011-1  ┆ 228                │
│ 19908011-2  ┆ 228                │
│ 19917963-8  ┆ 124                │
│ 19917963-7  ┆ 124                │
│ 19917963-15 ┆ 124                │
│ 19917963-10 ┆ 124                │
│ 19917963-23 ┆ 124                │
│ 19917963-1  ┆ 124                │
└─────────────┴────────────────────┘

📋 EJEMPLO DE REGISTROS DUPLICADOS:
CUM: 19903212-1 (232 registros)


CUM,EXPEDIENTE CUM,CONSECUTIVO,ESTADO REGISTRO,CANTIDAD CUM,ESTADO CUM,MUESTRA MÉDICA,UNIDAD,ATC,DESCRIPCIÓN_ATC,VÍA ADMINISTRACIÓN,CONCENTRACIÓN,PRINCIPIO ACTIVO,UNIDAD MEDIDA,CANTIDAD,UNIDAD REFERENCIA,FORMA FARMACÉUTICA
str,i64,i64,str,f64,str,str,str,str,str,str,str,str,str,f64,str,str
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""VITAMINA B12 COMO CIANOCOBALAM…","""æg""",0.6,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""VITAMINA D COMO VITAMINA D3""","""IU""",20.0,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""MAGNESIO COMO MAGNESIO CLORURO""","""g""",0.02,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""VITAMINA B2 COMO RIBOFLAVINA S…","""æg""",170.0,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""ZINC COMO SULFATO DE ZINC""","""g""",0.00112,"""100ML""","""SOLUCION ORAL"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""CLORO COMO CLORURO DE POTASIO.…","""g""",0.124,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""CLORO COMO CLORURO DE POTASIO.…","""g""",0.124,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""CARBOHIDRATOS""","""g""",13.72,"""100ML""","""SOLUCION ORAL"""


In [14]:
# Filtrar registros con CUM específico
registros_cum_especifico = df_preproc.filter(pl.col('CUM') == "19903212-1")

print(f"📋 REGISTROS CON CUM = '19903212-1':")
print(f"Total registros encontrados: {len(registros_cum_especifico):,}")

registros_cum_especifico.unique()

📋 REGISTROS CON CUM = '19903212-1':
Total registros encontrados: 232


CUM,EXPEDIENTE CUM,CONSECUTIVO,ESTADO REGISTRO,CANTIDAD CUM,ESTADO CUM,MUESTRA MÉDICA,UNIDAD,ATC,DESCRIPCIÓN_ATC,VÍA ADMINISTRACIÓN,CONCENTRACIÓN,PRINCIPIO ACTIVO,UNIDAD MEDIDA,CANTIDAD,UNIDAD REFERENCIA,FORMA FARMACÉUTICA
str,i64,i64,str,f64,str,str,str,str,str,str,str,str,str,f64,str,str
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""MANGANESO COMO MANGANESO SULFA…","""æg""",248.0,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""CLORO COMO CLORURO DE POTASIO.…","""g""",0.124,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""VITAMINA K""","""æg""",4.0,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""COBRE COMO SULFATO CUPRICO""","""g""",0.0001,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""CARBOHIDRATOS""","""g""",13.72,"""100ML""","""SOLUCION ORAL"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""MOLIBDENO COMO MOLIBDATO DE SO…","""æg""",7.6,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""POTASIO COMO POTASIO CLORURO. …","""g""",0.148,"""100ML""","""SOLUCION ORAL"""
"""19903212-1""",19903212,1,"""Vencido""",240.0,"""Inactivo""","""No""","""U""","""A11AA04""","""MULTIVITAMINAS Y ELEMENTOS TRA…","""ORAL""","""B""","""VITAMINA B12 COMO CIANOCOBALAM…","""æg""",0.6,"""100ML""","""SOLUCION ORAL"""


In [16]:
df_preproc_unicos = df_preproc.unique()
df_preproc_unicos.write_parquet("data/medicamentos_preprocesados.parquet")
df_preproc_unicos


CUM,EXPEDIENTE CUM,CONSECUTIVO,ESTADO REGISTRO,CANTIDAD CUM,ESTADO CUM,MUESTRA MÉDICA,UNIDAD,ATC,DESCRIPCIÓN_ATC,VÍA ADMINISTRACIÓN,CONCENTRACIÓN,PRINCIPIO ACTIVO,UNIDAD MEDIDA,CANTIDAD,UNIDAD REFERENCIA,FORMA FARMACÉUTICA
str,i64,i64,str,f64,str,str,str,str,str,str,str,str,str,f64,str,str
"""20167292-1""",20167292,1,"""Vigente""",30.0,"""Activo""","""No""","""U""","""J05AR26""","""DARUNAVIR Y RITONAVIR""","""ORAL""","""A""","""DARUNAVIR ETANOLATO 867.6 MG. …","""mg""",800.0,"""UNA TABLETA RECUBIERTA""","""TABLETA CUBIERTA CON PELICULA"""
"""20015204-5""",20015204,5,"""Vigente""",1.0,"""Activo""","""No""","""U""","""A11AA01""","""MULTIVITAMINAS MAS HIERRO""","""ORAL""","""A""","""DL-ALFA TOCOFERIL ACETATO / VI…","""mg""",10.0,"""""","""CÁPSULA BLANDA DE GELATINA CON…"
"""20061748-3""",20061748,3,"""Vigente""",1.0,"""Inactivo""","""No""","""U""","""B05BA10""","""COMBINACIONES""","""PARENTERAL""","""C""","""L-SERINA""","""g""",6.5,"""1000ML DE SOLUCIÓN""","""EMULSION INYECTABLE"""
"""20061746-8""",20061746,8,"""Vigente""",1.0,"""Inactivo""","""Si""","""U""","""B05BA10""","""COMBINACIONES""","""INTRAVENOSA""","""C""","""L-TREONINA""","""mg""",2.2,"""ML""","""EMULSION INYECTABLE"""
"""20009806-9""",20009806,9,"""Temp. no comerc - Vigente""",50.0,"""Activo""","""No""","""U""","""N02BE51""","""PARACETAMOL COMBINACIONES EXCL…","""ORAL""","""A""","""CAFEINA ANHIDRA""","""mg""",30.0,"""CAPSULA""","""CAPSULA DURA"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""20048292-2""",20048292,2,"""Abandono""",10.0,"""Inactivo""","""No""","""U""","""C10AA07""","""ROSUVASTATINA""","""ORAL""","""A""","""ROSUVASTATINA CÁLCICA 20.79MG …","""mg""",20.0,"""CADA TABLETA RECUBIERTA CONTIE…","""TABLETA RECUBIERTA"""
"""19927508-2""",19927508,2,"""Vencido""",120.0,"""Inactivo""","""No""","""U""","""R06AB04""","""CLORFENAMINA""","""ORAL""","""B""","""CLORFENIRAMINA MALEATO""","""g""",0.04,"""JARABE""","""JARABE"""
"""19952712-1""",19952712,1,"""Vigente""",100.0,"""Activo""","""No""","""U""","""H03BA02""","""PROPILTIOURACILO""","""ORAL""","""A""","""PROPILTIOURACILO""","""mg""",100.0,"""TABLETA""","""TABLETA"""
